In [1]:
import sys
sys.path.append('..')

In [2]:
from nbhd import data

In [3]:
db = data.Base()

Initializing database connection...
Database connected!


In [4]:
from xy import xy

In [5]:
from shapely.geometry import Point

In [8]:
from nbhd.geometry import pointbox

In [11]:
geom = pointbox(Point(xy),1000)

In [13]:
get_communities(db, geom)

NameError: name 'self' is not defined

In [12]:
def get_communities(
                    db,
                    geom,
                    footprint_threshold=250,
                    res_length_threshold=20,
                    short_threshold=20,
                    min_community_size=0,
                    node_distance=5):

   # nearest roads ~ 'slimroads' is without motorways and secondary roads
   # nearest buildings

    # 1 eliminate non-building properties : distance to building must == 0
    df1 = df.loc[df.dist_building == 0].copy()

    # 2 eliminate non-residential buildings
    building_counts = dict(df1.id_building.value_counts())
    df1['building_counts'] = df1.id_building.apply(
        lambda x: building_counts.get(x, 0))
    df1['footprint_area'] = gpd.GeoSeries(df1.geometry_building).area
    df1['footprint_area_per_uprn'] = df1.footprint_area / df1.building_counts
    df1['residential_building'] = df1[
        'footprint_area_per_uprn'] < footprint_threshold
    df2 = df1.loc[df1.residential_building].copy()

    # 3 establish whether roads are residential
    street_counts = dict(df2.id_street.value_counts())
    df2['street_counts'] = df2.id_street.apply(
        lambda x: street_counts.get(x, 0))
    df2['street_length_per_uprn'] = df2.length / df2.street_counts
    df2['residential_street'] = df2.street_length_per_uprn < res_length_threshold
    residential = dict(zip(df2.id_street, df2.residential_street))
    df['residential'] = df.id_street.apply(
        lambda x: residential.get(x, False))
    df['short_street'] = df.length < short_threshold
    df['res_or_short'] = df.residential | df.short_street
    df3 = df.loc[df.res_or_short].copy()

    # 4 treat nearby nodes as equivalent
    translator = self.db.get_nearest_nodes_translator(
        self.geom, node_distance)
    edges = df3.loc[~df3.duplicated()].copy()
    edges['translated_start'] = edges.startNode.apply(
        lambda x: translator.get(x, x))
    edges['translated_end'] = edges.endNode.apply(
        lambda x: translator.get(x, x))

    # 5 find connected networks of residential streets
    g = nx.from_pandas_edgelist(edges, 'translated_start',
                                'translated_end', True)
    subgraphs = [g.subgraph(c) for c in nx.connected_components(g)]
    sgs = [sg for sg in subgraphs if len(sg) > min_community_size]

    # 6 add community labels
    communities = dict()
    for i in range(len(sgs)):
        communities[str(i).zfill(2)] = list(
            nx.get_edge_attributes(sgs[i], 'id_street').values())
    communities_key = {
        value: key
        for key, value_list in communities.items() for value in value_list
    }

    df['community'] = df.id_street.apply(
        lambda x: communities_key.get(x, None))
    self.df = df
    return